# Lösungen zu den Zusatzübungen zum Notebook "Web Scraping"

☝️ Beachte: Es gibt beim Programmieren fast immer verschiedene Lösungswege. Deine Lösung mag anders aussehen, aber dennoch zum gewünschten Resultat führen. Das richtige Resultat ist das Wichtigste. 

1. Der in der Code-Zelle gegebene HTML-Code soll folgende Ausgabe erzeugen:

    ![](../../3_Dateien/Grafiken_und_Videos/HTML_Code.png)

    Aktuell fehlen im `<body>`-Element noch einige Tags. Überleg Dir mithilfe des Bildes, welche das sein könnten und an welchen Stellen sie fehlen. Ergänz sie in den bereits gegebenen `<>`. Du kannst Dir den vervollständigten Code z. B. auf [dieser Website](https://www.sejda.com/de/html-to-pdf) als PDF anzeigen lassen, um ihn mit dem Screenshot abzugleichen.

2. Die Website [manova.news](https://manova.news) (ehemals *Rubikon*, [siehe Wikipedia-Eintrag](https://de.wikipedia.org/wiki/Rubikon_(Website))) versteht sich als Nachrichtenportal, verbreitet aber häufig Verschwörungstheorien. Wir wollen die Sprache der Webseite genauer unter die Lupe nehmen. Dazu konzentrieren wir uns zunächst auf Schlagzeilen bzw. Überschriften in der Rubrik [*Natur und Mitwelt*](https://www.manova.news/section/21). 

    Verschaff Dir einen Überblick über die Seite und überleg Dir, worauf Du beim Scraping *aller* Schlagzeilen bzw. Überschriften in dieser Rubrik achten musst. Schau dazu im Quelltext nach. Notier Deine Antwort in der folgenden Code-Zelle.
    
    Halt außerdem stichpunktartig fest, wie Du beim Abruf- und Extraktionsschritt vorgehen willst.

In [ ]:
'''Worauf muss geachtet werden?
Pagination: Die Beiträge sind auf mehrere Unterseiten verteilt und es gibt keine Möglichkeit, alle Beiträge 
auf einer Seite anzuzeigen. Folglich müssen die Seiten nacheinander abgerufen werden, um wirklich alle Schlagzeilen 
extrahieren zu können.'''

'''Vorgehen beim Abrufen und Extrahieren:
1. Im Seitenquelltext herausfinden, wie man von einer Seite auf die jeweils nächste Seite gelangt. Dabei fällt auf, 
dass im Quelltext jeweils nur die Linkendung angegeben ist, welche noch um den Stammlink ergänzt werden muss 
(vgl. Notebook "Web Scraping Teil 1", Übung 1).
2. Coden einer Kontrollstruktur, die die aktuelle Seite scrapt, den Link zur nächsten Seite extrahiert und auf diese
Weise alle Seiten bis zur letzten nacheinander scrapt. Anhängen aller Quelltexte an eine Liste.
3. Extrahieren der Headlines, Orientierung am Tag <h2>.'''

3. Ruf nun den Quelltext aller Unterseiten der gegebenen Rubrik ab und speicher deren Quelltexte in einer Liste. Verwend hier wenn möglich einen regulären Ausdruck. Lass Dir anschließend in der zweiten Code-Zelle zur Kontrolle die ersten 1000 Zeichen des ersten Quelltexts ausgeben.

In [ ]:
#Abrufschritt mit Nutzung regulärer Ausdrücke

import re, requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

"""Definieren eines regulären Ausdrucks, um den Link zur jeweils nächsten Seite zu extrahieren;
Runde Klammern umschließen Gruppe mit eigentlichem Link; '\S+' matcht ein oder mehrere Zeichen, 
die nicht whitespace sind (vgl. Notebook "Reguläre Ausdrücke")"""
regex = r'<a class="button secondary" rel="next" href="(\S+)</a>' 

#Definieren von Stammlink, erster Linkendung sowie des kompletten Links zur ersten zu scrapenden Seite
base_link = "https://www.manova.news"
link_ending = "/section/21"
link = base_link + link_ending

all_pages = [] #Initialisieren einer leeren Liste, an die unten die einzelnen Quelltexte gehängt werden

"""'while'-Schleife scrapt eine paginierte Seite nach der anderen, bis kein Link mehr auf eine nächste Seite
im Quelltext gefunden wird"""
while True:
    
    print(f"Aktuell wird die Seite {link} gescrapt.") #Ausgabe des Fortschritts
    
    current_page = requests.get(link, timeout=5, headers=headers) #Abruf des Quelltexts zum aktuellen Link
    
    """Definieren des Encodings des Quelltexts ('requests' geht unspezifiziert vom falschen Encoding aus, 
    was sich z. B. an Umlauten zeigt)"""
    current_page.encoding = "UTF-8" 
    
    #Anfügen des eigentlichen Quelltexts der aktuellen Seite (Zugriff über 'text'-Attribut) an 'all_pages'
    all_pages.append(current_page.text) 
    
    #Quelltext nach regulärem Ausdruck absuchen, der den Link zur nächsten Seite matcht
    next_page = re.search(regex, current_page.text)
    
    #Wenn ein match gefunden wird...
    if next_page:
        """...Definieren des Links für die nächste Seite, indem wir mithilfe der 
        'group'-Methode auf die erste Gruppe zugreifen und die neue Linkendung an den Stammlink hängen."""
        link = base_link + next_page.group(1)
    else: #Wenn nicht, ist die Rubrik komplett gescrapt und die 'while'-Schleife wird abgebrochen.
        break

In [ ]:
"""Abrufschritt ohne die Nutzung regulärer Ausdrücke: Dieser Lösungsvorschlag enthält eine hardgecodete Abrufschleife, 
was die Übertragbarkeit des Codes etwa auf andere Rubriken der Webseite einschränkt. Um das Hardcoding zu vermeiden 
und trotzdem keine regulären Ausdrücke einzusetzen, könnten wir auch den jeweiligen Quelltext mit 'BeautifulSoup' parsen 
und die 'find'-Methode verwenden, um das Element mit dem Link auf die jeweils nächste Seite daraus zu extrahieren."""

import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

all_pages = [] #An die Liste werden anschließend die Quelltexte angehängt

#hardgecodete Zählschleife mit 'range' von 1 bis 9 (letzte Seitenzahl plus eins, da Endindex exklusive)
for number in range(1,9):
    link = f"https://www.manova.news/section/21?page={number}" #Nutzung eines f-strings, da sich bei der Linksendung nur die Seitenzahl ändert.
    
    print(f"Aktuell wird die Seite {link} gescrapt.")
    
    current_page = requests.get(link, timeout=5, headers=headers) #Abruf des Quelltexts zum aktuellen Link
    
    """Definieren des Encodings des Quelltexts ('requests' geht unspezifiziert vom falschen Encoding aus, 
    was sich z. B. an Umlauten zeigt)"""
    current_page.encoding = "UTF-8" 
    
    #Anfügen des eigentlichen Quelltexts der aktuellen Seite (Zugriff über 'text'-Attribut) an 'all_pages'
    all_pages.append(current_page.text) 

In [ ]:
#Ausgabe der ersten 1000 Zeichen des ersten Quelltexts (mit Index null)
print(all_pages[0][0:1000]) 

4. Extrahier nun alle Schlagzeilen bzw. Überschriften der angeteaserten Artikel und lass sie Dir ausgeben.  

In [ ]:
from bs4 import BeautifulSoup

for page in all_pages: #Iterieren über die Liste 'all_pages'
    
    soup = BeautifulSoup(page) #Initialisieren eines 'BeautifulSoup'-Objekts mit dem jeweiligen Quelltext
    
    #Da alle Überschriften im <body>-Element stehen, können wir unser 'BeautifulSoup'-Objekt "maßschneidern"
    body = soup.find("body")
    
    #Iterieren über eine Liste mit allen gefundenen <h2>-Elementen, also allen Schlagzeilen bzw. Überschriften
    for header in body.find_all("h2"):
        print(header.text.strip()) #Ausgabe des Textinhalts jedes Elements, bereinigt von whitespace

5. Da Web Scraping die angerufenen Server immer belastet, sollten wir so selten wie möglich Inhalte herunterladen. Wenn wir mehrfach die gleichen Daten für ein größeres Projekt benötigen, sollten wir sie nur einmal herunterladen und anschließend lokal speichern. 

    Die Quelltexte der manova-Seiten sind momentan nur in unserem Arbeitsspeicher (in der Variablen `all_pages`). Sobald wir diese Sitzung (also den Kernel) beenden, gehen sie verloren. Um auch in der nächsten Sitzung mit den Daten arbeiten zu können (ohne sie erneut vom Server herunterladen zu müssen), ist es nun Deine Aufgabe, jeden Quelltext in einem separaten HTML-Dokument zu speichern. Wähl als Speicherort das Verzeichnis "3_Dateien/Output".
    
    Überprüf abschließend über Deinem Dateimanager, ob die neu geschaffenen Dateien existieren sowie sinnvoll beschrieben wurden.

In [ ]:
#Iteration über 'all_pages' mithilfe von 'range', um einen Zähler für die Dateibenennung zu erhalten
for i in range(len(all_pages)):
    
    #Anlegen und Öffnen einer neuen Datei mit 'i' im Dateinamen und ".html" als Endung
    with open(f"../../3_Dateien/Output/manova_{i+1}.html", "w") as write_file:
        
        #Schreiben des über 'i' indizierten Quelltexts auf 'all_pages' in das geöffnete 'write_file'
        write_file.write(all_pages[i])

#Überprüfen, ob das Speichern erfolgreich war, mittels Dateimanager und eines Editors (z. B. Sublime Text)

6. Nun wollen wir mit den soeben lokal gespeicherten HTML-Dokumenten weiterarbeiten. Lies sie nacheinander ein und extrahier bei jedem angeteaserten Artikel das zugehörige Schlagwort. Beim Artikel im Screenshot unten wäre dies "#WASSERSPEZIAL" (andere Schlagworte beginnen nicht mit einem Hashtag).
  
    ![](../../3_Dateien/Grafiken_und_Videos/schlagwort_manova.png)

    Ermittle, wie häufig jedes Schlagwort auf den gescrapten Seiten zum Einsatz kommt. Lass Dir das Ergebnis in absteigender Reihenfolge ausgeben.

In [ ]:
import os #Import von 'os', um über die Dateien im Verzeichnis "3_Dateien/Output" zu iterieren

path = "../../3_Dateien/Output/" #Pfad zum Verzeichnis

freq_dict = {} #Initialisieren eines dictionaries, um Schlagworte sowie ihr Vorkommen zu speichern

#Iteration über alle Dateien im angegebenen Verzeichnis
for file in os.listdir(path):
    
    #Falls 'file' nicht mit "manova" beginnt, überspringen wir die Datei
    if not file.startswith("manova"):
        continue
        
    #Öffnen der jeweiligen HTML-Datei durch Konkatenieren von 'path' und 'file'
    with open(path + file) as f:
        
        #Parsen des geöffneten Quelltexts mit 'BeautifulSoup'
        soup = BeautifulSoup(f.read())
    
    """Iterieren über eine Liste mit allen Schlagworten, die wir über eine Kombination 
    von Tag (<div>) und Attribut 'class="tag"' finden können"""
    for tag in soup.find_all("div", class_="tag"):
        
        #Falls das im Textinhalt versteckte Schlagwort noch kein Schlüssel in 'freq_dict' ist...
        if not tag.text in freq_dict:
            freq_dict[tag.text] = 1 #legen wir den Schlüssel an und setzen den Wert auf eins
        else: #ansonsten erhöhen wir den Wert beim entsprechenden Schlüssel um eins
            freq_dict[tag.text] +=1

#Sortieren von 'freq_dict' nach Werten             
freq_dict_sorted = sorted(freq_dict.items(), key = lambda x: x[1], reverse=True)

#Iteration über 'freq_dict_sorted' zur Ausgabe von Schlagworten und Vorkommen
for tag, frequency in freq_dict_sorted:
    print(tag, "kommt", frequency, "Mal vor.")

7. Stell analog zu Übung 6 eine Übersicht über die fleißigsten Autor:innen in dieser Rubrik zusammen. Lass Dir die Namen derer ausgeben, die mehr als einen Artikel verfasst haben.

In [ ]:
#erster Teil des Codes analog zu Übung 6, Kommentare s. o.
path = "../../3_Dateien/Output/" 

freq_dict = {} 

for file in os.listdir(path):
    
    if not file.startswith("manova"):
        continue
        
    with open(path + file) as f:
        
        soup = BeautifulSoup(f.read())
    
    """Iterieren über eine Liste mit allen Elementen mit dem Tag <div> und dem Attribut
    'class=article-meta', in deren untergeordnetem <a>-Element sich der Name der jeweiligen 
    Autor:in verbirgt"""
    for meta_info in soup.find_all("div", class_="article-meta"):
        
        author = meta_info.find("a").text #Extrahieren des Textinhalts des (einzigen) untergeordneten <a>-Elements
        
        #optionale Bereinigung der Namen von whitespace (etwa zwischen Vor- und Nachnamen)
        import re
        author = re.sub(r"\s+", " ", author)        
        
        #Code ab hier bis zur Ausgabe am Ende analog zu Übung 6, Kommentare s. o.
        if not author in freq_dict:
            freq_dict[author] = 1
        else:
            freq_dict[author] +=1
            
freq_dict_sorted = sorted(freq_dict.items(), key = lambda x: x[1], reverse=True)

for author, frequency in freq_dict_sorted:
    
    #Überspringen von Autor:innen, die nur einen Artikel verfasst haben
    if frequency == 1:
        continue
        
    print(author, "schrieb", frequency, "Artikel.")

8. Bislang haben wir nur die Überblicksseiten der Rubrik "Natur und Mitwelt" gescrapt. Nun wollen wir die eigentlichen Artikeltexte scrapen und daraus ein eigenes Korpus erstellen. 

    Extrahier dazu zunächst alle Links auf den Überblicksseiten, die zu den eigentlichen Artikeln führen. Speichere die *vollständigen* Links in einer Liste.

In [ ]:
#erster Teil des Codes analog zu Übung 6, Kommentare s. o.
path = "../../3_Dateien/Output/" 

links = [] #Initialisieren einer Liste, an die wir unten alle Links anhängen

for file in os.listdir(path):
    
    if not file.startswith("manova"):
        continue
        
    with open(path + file) as f:
        
        soup = BeautifulSoup(f.read())
     
    """Iterieren über eine Liste mit allen Elementen mit dem Tag <h2>, in deren untergeordnetem <a>-Element 
    sich der Link zum jeweiligen Artikel im 'href'-Attribut verbirgt"""
    for header in soup.find_all("h2"):
        
        links.append(header.find("a").get("href")) #Extraktion des Werts (d. h. des Links)

#Vervollständigung der Links, d. h. Ergänzung um Stammlink mittels List Comprehension
links = [f"https://www.manova.news/{link}" for link in links]

print(links)

9. Lade für unser Korpus in der ersten Code-Zelle den Quelltext sämtlicher Artikel, deren Links Du eben zusammengetragen hast, herunter. Da wir die relevanten Teile davon gleich lokal speichern werden, kannst Du die Quelltexte zwischenzeitlich in Deinem Arbeitsspeicher belassen.

    Extrahier anschließend in der zweiten Code-Zelle aus jedem Quelltext den **Haupttext** des Artikels. Speichere alle Texte in *einer* XML-Datei im Ordner "3_Dateien/Output". Das XML-Dokument wird unserem Korpus entsprechen. Leg dessen hierarchische Struktur so an, dass die Artikeltexte voneinander unterschieden werden können.

In [ ]:
#Abrufschritt
from tqdm import tqdm #Import des Moduls für die Fortschrittsanzeige

source_codes = [] #Erstellen einer leeren Liste, an die die Quelltexte unten gehängt werden

#Iteration über 'links' mithilfe von 'range', um einen Zähler für die Dateibenennung zu erhalten; Fortschrittsanzeige mithilfe von 'tqdm'
for i in tqdm(range(len(links))):
    """Verwenden von 'trafilatura', das zur Extraktion von Haupttexten wunderbar geeignet ist"""
    source_codes.append(trafilatura.fetch_url(links[i])) #Indizieren des jeweiligen Links auf 'links' mit 'i'

In [ ]:
#Extraktions- und Speicherschritt
import xml.etree.ElementTree as ET #Import der benötigten XML-Module
from xml.dom import minidom

corpus = ET.Element("corpus") #Anlegen des obersten Elements in der XML-Hierarchie

#Iteration über 'source_codes', mit Fortschrittsanzeige
for source_code in tqdm(source_codes):

    main_text = trafilatura.extract(source_code) #Extraktion des Haupttexts
    
    """Für jeden Artikel schaffen wir ein Element namens 'article' (zweites Argument der 'SubElement'-Funktion), 
    das dem 'corpus'-Element untergeordnet sein soll (erstes Argument). Im Hinblick auf die Herausforderung:
    Wenn Du das Korpus mit weiteren (Meta-)Daten anreichern möchtest (für deren Extraktion wie gesagt 'requests' und 
    'BeautifulSoup' nötig wären), dann könntest Du diese Informationen mittels der 'set'-Methode als Attribute von 
    'article' festlegen. Hier belassen wir es beim Artikeltext."""
    article = ET.SubElement(corpus, "article") 
    
    #Iteration über die einzelnen Paragraphen des jeweiligen Artikeltexts
    for paragraph in main_text.split("\n"):
        
        """Für jeden Paragraphen des jeweiligen Artikels schaffen wir mithilfe derselben 'SubElement'-Funktion 
        ein Element namens 'p' (zweites Argument), das dem jeweiligen 'article'-Element untergeordnet sein soll 
        (erstes Argument). Anschließend weisen wir 'p' Textinhalt über das Attribut 'text' zu."""
        paragraph_xml = ET.SubElement(article, "p")
        paragraph_xml.text = paragraph
        
#Schön formatieren, d. h. Einfügen von Einrückungen
pretty_corpus = minidom.parseString(ET.tostring(corpus)).toprettyxml(indent="  ")

#Speichern der XML-Datei
with open("../../3_Dateien/Output/manova_corpus.xml", "w", encoding="utf-8") as write_file:
    write_file.write(pretty_corpus)

10. Auch wenn es beim Web Scraping nur um die Daten*beschaffung* geht, so bietet folgender vorgegebener Code einen Einblick in die Daten*auswertung*, die möglich ist, jetzt wo wir unser eigenes Korpus zusammengestellt haben. Der Code lässt sich ausführen, wenn Du die XML-Elemente gleich genannt hast, wie in der Lösung vorgeschlagen. Du kannst ihn natürlich an Deine Namensgebung anpassen. 

    Wir verwenden das im Notebook "Funktionen und Methoden Teil 2" erstellte Modul `keywords` bzw. die Funktion `get_freqs` daraus. Diese haben wir ja geschrieben, um die Häufigkeit von Wörtern zu berechnen. Da wir es nun mit einer viel größeren Datenmenge zu tun haben, läuft der Code eine Weile. Bau gerne eine Fortschrittsanzeige mit `tqdm` in den Modulcode, um die Ausführung im Blick behalten zu können. Bedenk aber, dass Du nach dem Ändern des Modulcodes den Kernel neustarten musst, um das Modul mit dem geänderten Code zu importieren. 
    
    Behalt auch im Hinterkopf, dass wir diesen Code in einem früheren Stadium unserer Programmierkarriere geschrieben haben. Mittlerweile kennen wir vermutlich effizientere Wege, um Wörter auszuzählen. Ungeachtet dessen offenbart sich hier (einmal mehr) der Vorteil von *Code Reuse* und konkret von Modularisierung, wie er im zweiteiligen Notebook "Funktionen und Methoden" aufgezeigt wurde.

In [ ]:
#Einlesen des eben erstellen XML-Dokuments
tree = ET.parse("../../3_Dateien/Output/manova_corpus.xml") #Achtung: anderer Pfad als im Lehrnotebook
root = tree.getroot() 

text = "" #Initialisieren eines leeren strings, an den wir unten die Artikeltexte hängen

#Iteration (rekursiv!) über alle <p>-Elemente und Anhängen des Textinhalts an 'text'
for paragraph in root.iter("p"):
    text += paragraph.text
    
#Einlesen der Stoppwörter, die wir auch im Notebook "Funktionen und Methoden Teil 2" benutzt haben
stopwords_list = []
with open("../../3_Dateien/Koalitionsvertraege/stopwords-de.txt", encoding="utf-8") as h: #Achtung: anderer Pfad als im Lehrnotebook
    for line in h:
        stopwords_list.append(line.rstrip())

#Hinzufügen eines Verzeichnis, in dem Python nach Modulen sucht
import sys
sys.path.append("../../3_Dateien/Module") #Achtung: anderer Pfad als im Lehrnotebook

#Importieren der Funktion 'get_freqs'
from keywords import get_freqs

#Berechnen der Worthäufigkeiten
word_frequencies = get_freqs(text, stopwords_list)

#Ausgabe der zehn häufigsten Wörter
print("Die zehn häufigsten Wörter (exkl. Stoppwörter) sind...")
for word, frequency in word_frequencies[0:10]:
    print(word, "kommt", frequency, "vor.")

***
<table>
      <tr>
        <td>
            <img src="../../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>